In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

In [4]:
data = pd.read_csv("E:\SWJP\ML_Brain\Data_Port\water_potability.csv")

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Turjo\AppData\Local\Temp\ipykernel_31372\1959279339.py:1: SyntaxWarning: invalid escape sequence '\S'
  data = pd.read_csv("E:\SWJP\ML_Brain\Data_Port\water_potability.csv")


In [5]:
data

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
...,...,...,...,...,...,...,...,...,...,...
3271,4.668102,193.681735,47580.991603,7.166639,359.948574,526.424171,13.894419,66.687695,4.435821,1
3272,7.808856,193.553212,17329.802160,8.061362,NaN,392.449580,19.903225,NaN,2.798243,1
3273,9.419510,175.762646,33155.578218,7.350233,NaN,432.044783,11.039070,69.845400,3.298875,1
3274,5.126763,230.603758,11983.869376,6.303357,NaN,402.883113,11.168946,77.488213,4.708658,1


In [5]:
data_imputed = data.copy()
for col in ['ph', 'Sulfate', 'Trihalomethanes']:
    data_imputed[col].fillna(data_imputed[col].median(), inplace=True)

C:\Users\Turjo\AppData\Local\Temp\ipykernel_16292\1474709023.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_imputed[col].fillna(data_imputed[col].median(), inplace=True)
C:\Users\Turjo\AppData\Local\Temp\ipykernel_16292\1474709023.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

In [6]:
X = data_imputed.drop('Potability', axis=1)
y = data_imputed['Potability']

In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

C:\Users\Turjo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [10]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machine': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'Gradient Boosting': GradientBoostingClassifier()
}

results = {}
reports = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    reports[name] = classification_report(y_test, y_pred)

In [11]:
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
print("Model Performance:")
for model, acc in sorted_results:
    print(f"{model}: {acc:.2f}")


Model Performance:
Random Forest: 0.72
K-Nearest Neighbors: 0.67
Support Vector Machine: 0.66
Decision Tree: 0.65
Gradient Boosting: 0.64
Naive Bayes: 0.55
Logistic Regression: 0.51


In [12]:
svm_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']}
svm_model = SVC()

grid_search = GridSearchCV(svm_model, svm_params, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Evaluate the best model on the test set
best_svm = grid_search.best_estimator_
y_pred = best_svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("\nBest Model (Support Vector Machine) Performance:")
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_rep)


Best Model (Support Vector Machine) Performance:
Accuracy: 0.69

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.65      0.68       601
           1       0.67      0.73      0.70       598

    accuracy                           0.69      1199
   macro avg       0.69      0.69      0.69      1199
weighted avg       0.69      0.69      0.69      1199



In [13]:
import tkinter as tk
from tkinter import messagebox

def predict():
    try:
        # Retrieve input values
        ph = float(ph_entry.get())
        hardness = float(hardness_entry.get())
        solids = float(solids_entry.get())
        chloramines = float(chloramines_entry.get())
        sulfate = float(sulfate_entry.get())
        conductivity = float(conductivity_entry.get())
        organic_carbon = float(organic_carbon_entry.get())
        trihalomethanes = float(trihalomethanes_entry.get())
        turbidity = float(turbidity_entry.get())
        
        # Example: You can replace this section with your prediction logic
        features = [ph, hardness, solids, chloramines, sulfate, conductivity, organic_carbon, trihalomethanes, turbidity]
        # Replace the line below with your model's prediction code
        result = "Potable" if sum(features) % 2 == 0 else "Not Potable"  # Dummy prediction logic
        
        # Display the result
        messagebox.showinfo("Prediction Result", f"The water is: {result}")
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numeric values for all fields.")

root = tk.Tk()
root.title("Water Potability Prediction")

fields = [
    "ph", "Hardness", "Solids", "Chloramines", "Sulfate", 
    "Conductivity", "Organic Carbon", "Trihalomethanes", "Turbidity"
]

entries = {}
for idx, field in enumerate(fields):
    label = tk.Label(root, text=f"{field}:")
    label.grid(row=idx, column=0, padx=10, pady=5)
    entry = tk.Entry(root)
    entry.grid(row=idx, column=1, padx=10, pady=5)
    entries[field] = entry

ph_entry = entries["ph"]
hardness_entry = entries["Hardness"]
solids_entry = entries["Solids"]
chloramines_entry = entries["Chloramines"]
sulfate_entry = entries["Sulfate"]
conductivity_entry = entries["Conductivity"]
organic_carbon_entry = entries["Organic Carbon"]
trihalomethanes_entry = entries["Trihalomethanes"]
turbidity_entry = entries["Turbidity"]

predict_button = tk.Button(root, text="Predict", command=predict)
predict_button.grid(row=len(fields), column=0, columnspan=2, pady=10)

root.mainloop()
